In [118]:
# !pip install protobuf==3.20.3
# !pip install --upgrade tokenizers
# !pip install crewai crewai_tools langchain_community python-dotenv
# !pip install exifread
# !pip install chardet

In [127]:
# from google.colab import userdata
import os
from dotenv import load_dotenv
load_dotenv()

True

In [154]:
# input_dir = "original-pics"  # Replace with your input folder
# downscaled_dir = "downscaled-pics"  # Replace with your output folder
# selected_dir = "selected-pics"  # Replace with your output folder
# example_web_files = "example-markdown"

#use the below directories for local runtime
input_dir = "example-files/original-pics/"  # Replace with your input folder
downscaled_dir = "example-files/downscaled-pics/"  # Replace with your output folder
selected_dir = "example-files/selected-pics/"  # Replace with your output folder
example_web_files = "example-files/example-markdown/"
watermark_filepath = "example-files/logo_watermark-youtube.png"

In [155]:
from PIL import Image
import imghdr

def resize_images(input_folder, output_folder, max_dimension=1080):
    """
    Resizes images in the input folder, ensuring the largest dimension is max_dimension,
    maintaining aspect ratio, and saves them to the output folder, preserving date taken metadata.

    Args:
      input_folder: Path to the input folder containing images.
      output_folder: Path to the output folder where resized images will be saved.
      max_dimension: The maximum size for the largest dimension (width or height).
    """

    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file before checking image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with Image.open(image_path) as img:
                    # exif_data = img.getexif()  # Get EXIF data
                    # date_taken = exif_data.get(36867) if exif_data else None  # Extract date taken

                    width, height = img.size

                    # Calculate new dimensions based on the largest dimension
                    if width > height:
                        new_width = max_dimension
                        new_height = int(height * (max_dimension / width))
                    else:
                        new_height = max_dimension
                        new_width = int(width * (max_dimension / height))

                    resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)  # Use LANCZOS for better quality

                    # Preserve EXIF data in the resized image
                    # if date_taken:
                    resized_img.info['exif'] = img.info['exif']

                    output_path = os.path.join(output_folder, filename)
                    resized_img.save(output_path, exif=resized_img.info.get('exif'))  # Save with EXIF data
                    print(f"Resized {filename} and saved to {output_folder}")
            except Exception as e:  # Handle potential errors
                print(f"Error processing {filename}: {e}")


resize_images(input_dir, downscaled_dir)

Resized 20230406_193128-1.jpg and saved to example-files/downscaled-pics/
Resized P1020193.jpg and saved to example-files/downscaled-pics/
Resized P1070315.jpg and saved to example-files/downscaled-pics/
Resized P2170214.jpg and saved to example-files/downscaled-pics/
Resized P3200271.jpg and saved to example-files/downscaled-pics/


In [156]:
import base64

def images_to_base64_dict(input_folder):
    """
    Converts images in the input folder to a dictionary with image names as keys
    and their base64 encoded values as values.

    Args:
      input_folder: Path to the input folder containing images.

    Returns:
      A dictionary where keys are image filenames and values are base64 encoded strings.
    """

    image_dict = {}

    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        # Check if it's a file and a valid image format
        if os.path.isfile(image_path) and imghdr.what(image_path) in ('png', 'jpeg', 'gif', 'bmp'):
            try:
                with open(image_path, "rb") as image_file:
                    encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
                    image_dict[filename] = encoded_string
            except Exception as e:
                print(f"Error processing {filename}: {e}")

    return image_dict

image_dictionary = images_to_base64_dict(downscaled_dir)

# Print the dictionary (optional)
# print(image_dictionary)

In [157]:
agent_critic_background = """You are a highly respected art critic with 20 years of experience in the art market and 30 years
                          as a practicing artist. In your youth, you explored a wide range of artistic mediums,
                          from painting and sculpture to photography and performance art, giving you a deep understanding
                          of the creative process. Your passion for art has led you to travel the world,
                          immersing yourself in diverse cultures and artistic traditions. You possess a profound
                          knowledge of art history, from ancient cave paintings to contemporary installations,
                          and you are constantly seeking new knowledge through lectures, workshops, and exhibitions.
                          Your critical approach is characterized by a keen eye for visual storytelling,
                          an appreciation for the artist's process, and an openness to experimentation.
                          You are a master of articulating your thoughts in a clear and engaging manner,
                          providing insightful critiques that are both objective and constructive."""

agent_critic_task = """Please provide a detailed and objective analysis of the following image,
                        your analysis should include:
                      1. Objective Description:
                        - Provide a neutral and factual description of the image's subject matter, composition,
                            and key visual elements. Avoid subjective interpretations or emotional responses at this stage.
                        - Pay close attention to the use of color, line, form, texture, and space.
                      2. Critical Assessment:
                        - Your assessment will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.
                        - Technical Execution: Analyze the artist's technical skill in handling the chosen medium
                            and their mastery of fundamental artistic principles (e.g., perspective, proportion,
                            light and shadow).
                        - Composition and Design: Evaluate the effectiveness of the composition in guiding the
                            viewer's eye and creating visual interest. Consider the use of balance, rhythm,
                            and focal points.
                        - Color Palette: Discuss the artist's use of color, noting the dominant hues, contrasts,
                            and their impact on the overall mood and atmosphere.
                        - Conceptual Exploration: If applicable, delve into the conceptual underpinnings of the work.
                            Identify any symbolism, metaphors, or narratives that contribute to its meaning.
                        - Originality and Innovation: Comment on the originality of the work and whether it
                            demonstrates a unique approach or expands upon existing artistic conventions.¨
                      3. Overall Impression:
                        - Summarize your overall impression of the artwork, highlighting its strengths and weaknesses.
                        - Offer constructive suggestions for improvement, if applicable.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                      4. Objective Quality Score:
                        - Assign a score between 1 and 10 (10 being the highest) based solely on your 
                            assessment of the criteria above.
                        - It is crucial that your score is free from personal bias or preference. Focus on the
                            artwork's technical merits, compositional strength, and conceptual depth.
                        - If an image is not good, boring, or poor, you are not afraid to say so.
                        - Your score will be based on how good you believe others will find the art work. If
                            you yourself like a picture but you think others don't find it striking or interesting
                            the image will get a lower score.

                      Remember: Your role is to provide an objective and insightful critique, without aiming to please
                          your assessment is attachedd to your reputation, so you focus on being objective.
                          If an image is not good, boring, or poor, you are not afraid to say so."""

In [165]:
import litellm
import json
import time

critic_prompt = agent_critic_background + "\n\n" + agent_critic_task

def analyze_images(image_dictionary):
    """
    Analyzes images in a dictionary, sending them to Gemini for description,
    critical assessment, and quality score.

    Args:
        image_dictionary: A dictionary where keys are image names and values
                           are base64 encoded image strings.

    Returns:
        A dictionary containing analysis results for each image, following
        the defined schema.
    """
    # os.environ['GEMINI_API_KEY'] = userdata.get("GOOGLE_API_KEY")   #use this while in colab

    results = {}

    for image_name, base64_image in image_dictionary.items():
        try:
            response = litellm.completion(
                # model="gemini/gemini-1.5-pro",
                model="gemini/gemini-1.5-flash",
                temperature=0.7,
                max_tokens=4096,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": critic_prompt
                            },
                            {
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                            },
                        ],
                    }
                ],
                # api_key=userdata.get("GOOGLE_API_KEY"),  # Replace with your API key source if needed
                response_format={
                    "type": "json_object",
                    "response_schema": {
                        "type": "object",
                        "properties": {
                            "description": {"type": "string"},
                            "critical_assessment": {"type": "string"},
                            "quality_score": {"type": "integer", "minimum": 1, "maximum": 10},
                        },
                        "required": ["description", "critical_assessment", "quality_score"],
                    }
                }
            )

            # Attempt to decode the JSON response
            analysis = json.loads(response.choices[0].message.content) 
            
            # Check if the required keys are present in the analysis
            if all(key in analysis for key in ["description", "critical_assessment", "quality_score"]):
                results[image_name] = analysis
            else:
                print(f"Warning: Incomplete analysis for {image_name}. Missing required keys.")

        except (json.JSONDecodeError, KeyError, IndexError) as e:
            print(f"Error processing {image_name}: {e}")

        except Exception as e:  # Catching other potential exceptions
            print(f"Unexpected error processing {image_name}: {e}")

        time.sleep(2)  # Introduce a delay between API calls. not best practice but works for now

    return results

# Example usage:
# image_dictionary = {"image1.jpg": base64_image_string1, "image2.png": base64_image_string2, ...}
analysis_results = dict()
analysis_results_2 = dict()
analysis_results_3 = dict()
analysis_results = analyze_images(image_dictionary)
# analysis_results_2 = analyze_images(image_dictionary)
# analysis_results_3 = analyze_images(image_dictionary)
# analysis_results_4 = analyze_images(image_dictionary)
print(analysis_results)

{'20230406_193128-1.jpg': {'critical_assessment': "The photograph presents a straightforward depiction of three young men horseback riding in a desert landscape. While technically competent, the image lacks artistic depth or originality. The composition is functional rather than artistic, simply presenting the subjects in a fairly standard arrangement. The color palette is somewhat muted, with the earth tones dominating the scene. There's nothing particularly striking or innovative about this photograph, which will likely appeal only to those directly involved or with a specific interest in the subject matter. The technical execution is acceptable; the focus is sharp, and the exposure is well-managed. However, there is no particular artistic flair in the handling of the light or shadow. The photograph does not explore any deeper conceptual themes or narratives. It is a simple documentation of an event rather than an artistic statement.  The overall impression is that of a pleasant, but

In [160]:
#calculates avg score IF more than one art critique are done

for key, value in list(analysis_results.items()):  # Iterate over a copy of items
    try:
        scores = [
            value.get('quality_score'),  # From analysis_results
            analysis_results_2.get(key, {}).get('quality_score'),
            analysis_results_3.get(key, {}).get('quality_score')
            # analysis_results_4.get(key, {}).get('quality_score')
        ]
        
        if any(score is None or score == 0 for score in scores):
            print(f"Removing {key} due to invalid quality score (None or 0).")
            del analysis_results[key]  # Remove from original dictionary
            if key in analysis_results_2:
                del analysis_results_2[key]
            if key in analysis_results_3:
                del analysis_results_3[key]
            # if key in analysis_results_4:
            #     del analysis_results_4[key]
            continue  # Skip to the next iteration
        
        # If scores are valid, proceed with averaging and updating:
        avg_quality_score = sum(scores) / len(scores)
        analysis_results[key]['quality_score'] = avg_quality_score
        print(analysis_results[key]['quality_score'])
    except KeyError as e:
        print(f"KeyError encountered: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Removing 20230406_193128-1.jpg due to invalid quality score (None or 0).
Removing P1020193.jpg due to invalid quality score (None or 0).
Removing P1070315.jpg due to invalid quality score (None or 0).
Removing P2170214.jpg due to invalid quality score (None or 0).
Removing P3200271.jpg due to invalid quality score (None or 0).


In [166]:
# prompt: save dictionary to txt file, with some formatting or line spaces that make it easy to read

def save_dict_to_txt(data, filename):
  """Saves a dictionary to a text file with formatted output.

  Args:
    data: The dictionary to save.
    filename: The name of the file to save to.
  """
  try:
    with open(filename, 'w') as file:
      for key, value in data.items():
        file.write(f"{key}:\n\n")
        if isinstance(value, dict):
          for k, v in value.items():
            file.write(f"  {k}: {v}\n\n")
        elif isinstance(value, list):
            file.write("  [\n\n")
            for item in value:
                file.write(f"    {item},\n\n")
            file.write("  ]\n\n")
        else:
          file.write(f"  {value}\n\n")
        file.write("\n\n")  # Add an extra newline for better readability

  except Exception as e:
      print(f"An error occurred: {e}")

# Example usage (assuming 'analysis_results' is defined as in your original code)
# Replace with your actual dictionary and file name
save_dict_to_txt(analysis_results, "image_analysis_results.txt")

In [ ]:
# implemented changes:
# changing so after avg score is calculated and dict is saved to txt
# every image with quality score over 7.3 is moved to selected_dir
# then i am free to go into the directories and delete images, or add my own
# then filtered_results should be filtered not on 7.3 anymore, 
#     but the dictionary should be filtered to only contain the images that are currently in selected_dir

In [167]:
def move_high_quality_images(analysis_results, downscaled_dir, selected_dir, threshold=4.3):
    """Moves images with a quality score over the threshold to selected_dir.

    Args:
        analysis_results: The dictionary containing analysis results.
        downscaled_dir: The directory containing downscaled images.
        selected_dir: The directory to move selected images to.
        threshold: The quality score threshold for selecting images.
    """
    os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

    for filename, value in analysis_results.items():
        if value.get('quality_score', 0) > threshold:
            source_path = os.path.join(downscaled_dir, filename)
            destination_path = os.path.join(selected_dir, filename)

            if os.path.exists(source_path):
                shutil.move(source_path, destination_path)
                print(f"Moved {filename} to {selected_dir}")
            else:
                print(f"Warning: {filename} not found in {downscaled_dir}")

move_high_quality_images(analysis_results, downscaled_dir, selected_dir)

Moved P1020193.jpg to example-files/selected-pics/
Moved P1070315.jpg to example-files/selected-pics/
Moved P2170214.jpg to example-files/selected-pics/


In [168]:
def filter_results_in_selected_dir(analysis_results, selected_dir):
    """Filters the dictionary to only contain images currently in selected_dir.

    Args:
        analysis_results: The dictionary containing analysis results.
        selected_dir: The directory containing selected images.

    Returns:
        A filtered dictionary containing only images in selected_dir.
    """
    filtered_results = {}
    for filename in os.listdir(selected_dir):
        if filename in analysis_results:
            filtered_results[filename] = analysis_results[filename]
    return filtered_results


filtered_results = filter_results_in_selected_dir(analysis_results, selected_dir)
print(filtered_results)

{'20230406_193128-1.jpg': {'critical_assessment': "The photograph presents a straightforward depiction of three young men horseback riding in a desert landscape. While technically competent, the image lacks artistic depth or originality. The composition is functional rather than artistic, simply presenting the subjects in a fairly standard arrangement. The color palette is somewhat muted, with the earth tones dominating the scene. There's nothing particularly striking or innovative about this photograph, which will likely appeal only to those directly involved or with a specific interest in the subject matter. The technical execution is acceptable; the focus is sharp, and the exposure is well-managed. However, there is no particular artistic flair in the handling of the light or shadow. The photograph does not explore any deeper conceptual themes or narratives. It is a simple documentation of an event rather than an artistic statement.  The overall impression is that of a pleasant, but

In [ ]:
# prompt: filter the dictionary analysis_results to only keep elements where quality_score is equal to 8 or higher

# filtered_results = {key: value for key, value in analysis_results.items() if value.get('quality_score', 0) >= 7.3}
# filtered_results

{}

In [ ]:
# os.makedirs(selected_dir, exist_ok=True)  # Create the destination directory if it doesn't exist

# for filename in filtered_results:  # Iterate through the keys (filenames) in filtered_results
#     source_path = os.path.join(downscaled_dir, filename)
#     destination_path = os.path.join(selected_dir, filename)

#     if os.path.exists(source_path):
#         shutil.move(source_path, destination_path)
#         print(f"Moved {filename} to {selected_dir}")
#     else:
#         print(f"Warning: {filename} not found in {downscaled_dir}")

In [169]:
from pydantic import BaseModel, Field, TypeAdapter
from typing import Dict, Optional, List, Set, Tuple
import os
import exifread

class ImageAnalysisResult(BaseModel):
    filename: str = Field(..., description="Filename of the image.")
    description: str = Field(..., description="Objective description of what is in the image.")
    assessment: str = Field(..., description="Qualitative assessment of the image.")
    quality_score: float = Field(..., ge=1, le=10, description="Quality score of the image based on assessment.")
    date_taken: str = Field(..., description="Date the image was originally captured.")
    title: str = Field(None, description="Title of the image.")
    summary: str = Field(None, description="Publication ready and web friendly summary of the image.")


def convert_to_objects(filtered_results, downscaled_dir):
    image_objects = []
    image_adapter = TypeAdapter(ImageAnalysisResult)  # Create a TypeAdapter
    for filename, analysis in filtered_results.items():
        image_path = os.path.join(downscaled_dir, filename)
        try:
            # Using exifread to get date taken:
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f)
                date_taken = tags.get('EXIF DateTimeOriginal', tags.get('Image DateTime'))

                # Extract only the date if date_taken is not None:
                if date_taken:
                    date_taken_str = str(date_taken)
                    date_only = date_taken_str.split()[0]  # Split by space and take the first part (date)
                else:
                    date_only = None  # Handle cases where date_taken is None

            # Create ImageAnalysisResult object using TypeAdapter
            image_object = image_adapter.validate_python({
                "filename": filename,
                "description": analysis.get('description', ''),
                "assessment": analysis.get('critical_assessment', ''),
                "quality_score": analysis.get('quality_score', 0),
                "date_taken": date_only  # Use the extracted date
            })
            image_objects.append(image_object)
        except FileNotFoundError:
            print(f"Warning: Image file not found: {image_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    return image_objects

# Example usage (assuming filtered_results and downscaled_dir are defined)
image_objects = convert_to_objects(filtered_results, input_dir)  # Assuming 'input_dir' is your image directory

# Now you can access the attributes of the objects:
for obj in image_objects:
    print(f"Filename: {obj.filename}")
    print(f"Description: {obj.description}")
    print(f"Assessment: {obj.assessment}")
    print(f"Quality Score: {obj.quality_score}")
    print(f"Date Taken: {obj.date_taken}")
    print(f"Title: {obj.title}")  # Access the generated title
    print(f"Summary: {obj.summary}") # Access the generated summary
    print("-" * 20)

Filename: 20230406_193128-1.jpg
Description: The image shows three fair-skinned men of similar age and build astride three horses in a relatively flat desert area under a partly cloudy sky. The men are positioned slightly apart, with a central figure and one to either side. They appear to be looking directly at the camera. Each horse is a similar reddish-brown hue. The background includes low-lying, rocky hills under a pale blue sky with some cloud cover. The lighting suggests it is either early morning or late afternoon. The overall color palette is made up of muted earth tones, with browns, tans and muted blues dominating. The texture is relatively smooth, with the focus primarily on the subjects and horses. The space is relatively shallow, with the emphasis on the figures in the foreground. 
Assessment: The photograph presents a straightforward depiction of three young men horseback riding in a desert landscape. While technically competent, the image lacks artistic depth or original

In [137]:
# # prompt: move all images remaining in image_objects from downscaled_dir to selected_dir

# import shutil

# os.makedirs(selected_dir, exist_ok=True)

# for image_object in image_objects:
#     source_path = os.path.join(downscaled_dir, image_object.filename)
#     destination_path = os.path.join(selected_dir, image_object.filename)

#     if os.path.exists(source_path):
#         shutil.move(source_path, destination_path)
#         print(f"Moved {image_object.filename} to {selected_dir}")
#     else:
#         print(f"Warning: {image_object.filename} not found in {downscaled_dir}")

Moved G-P3050307.jpg to example-files/selected-pics/
Moved P1070391-1.jpg to example-files/selected-pics/
Moved P1070535.jpg to example-files/selected-pics/
Moved P2170075.jpg to example-files/selected-pics/
Moved P2170126.jpg to example-files/selected-pics/
Moved P3130066.jpg to example-files/selected-pics/
Moved P3200245.jpg to example-files/selected-pics/
Moved P3200268.jpg to example-files/selected-pics/
Moved P3200473.jpg to example-files/selected-pics/
Moved P3200514.jpg to example-files/selected-pics/


In [171]:

def watermark_images(selected_dir, watermark_filepath):
    watermark = Image.open(watermark_filepath)
    for filename in os.listdir(selected_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(selected_dir, filename)
            img = Image.open(filepath)
            width, height = img.size
            shortest_dim = min(width, height)
            watermark_width = int(0.18 * shortest_dim)
            watermark_height = int((watermark_width / watermark.width) * watermark.height)
            watermark = watermark.resize((watermark_width, watermark_height))
            padding = int(0.01 * shortest_dim)
            img.paste(watermark, (padding, height - watermark_height - padding), watermark)
            img.save(filepath)

watermark_images(selected_dir, watermark_filepath)

In [172]:

def organize_images(selected_dir):
    """
    Creates a new folder for each image in the selected directory,
    moves the image into its corresponding folder, renames it to "featured",
    and creates an index.md file.
    """
    for filename in os.listdir(selected_dir):
        name, ext = os.path.splitext(filename)
        if ext.lower() in ('.jpg', '.jpeg', '.png', '.gif', '.bmp'):
          try:
            # Create a new directory for the image
            new_dir = os.path.join(selected_dir, name)
            os.makedirs(new_dir, exist_ok=True)

            # Move and rename the image
            old_path = os.path.join(selected_dir, filename)
            new_path = os.path.join(new_dir, f"featured{ext}")
            os.rename(old_path, new_path)

            print(f"Moved and renamed '{filename}' to '{new_path}'")

            # Create index.md file
            index_md_path = os.path.join(new_dir, "index.md")
            with open(index_md_path, "w", encoding="utf-8") as f:
                f.write(f"# {name}\n\n")  # Example content, you can customize this
                # f.write("![featured](featured{ext})\n")

            print(f"Created 'index.md' in '{new_dir}'")

          except OSError as e:
              print(f"Error processing {filename}: {e}")

organize_images(selected_dir)

Moved and renamed '20230406_193128-1.jpg' to 'example-files/selected-pics/20230406_193128-1\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/20230406_193128-1'
Moved and renamed 'P1020193.jpg' to 'example-files/selected-pics/P1020193\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P1020193'
Moved and renamed 'P1070315.jpg' to 'example-files/selected-pics/P1070315\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P1070315'
Moved and renamed 'P2170214.jpg' to 'example-files/selected-pics/P2170214\featured.jpg'
Created 'index.md' in 'example-files/selected-pics/P2170214'


In [108]:
from crewai import Agent, Task, Process, Crew, LLM
from crewai_tools import DirectoryReadTool, FileReadTool, FileWriterTool

llm_general = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.4,
    max_tokens=4096,
    # api_key=userdata.get('GOOGLE_API_KEY')    #use when in colab
    # base_url="https://api.groq.com/openai/v1"
)

llm_developer = LLM(
    # model="gemini/gemini-1.5-pro",
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    max_tokens=4096,
    # api_key=userdata.get('GOOGLE_API_KEY')
    # base_url="https://api.groq.com/openai/v1"
)

In [109]:
critic_full_agent = Agent(
    llm=llm_general,
    role="Art Critic",
    goal="Analyse visual art and write compelling SEO content",
    backstory=agent_critic_background,
    verbose=True,
    memory=True,
    # output_pydantic=ImageAnalysisResult
)

# Define the web developer agent with tools
web_dev_agent = Agent(
    llm=llm_developer,
    role="Web Developer",
    goal="Organize images and create website structure",
    backstory="""You are an experienced web developer proficient in HTML, CSS, Markdown, and JavaScript.
                You are tasked with organizing image files and working on markdown files.
                Use the tools provided to interact with the file system.""",
    verbose=True,
    memory=True
)

In [110]:
read_examples_task = Task(
    description=f"""Read all the example files from the directory '{example_web_files}'.
                    Store the content of these files in your memory for later use.""",
    expected_output="""Example files read and stored in memory.""",
    agent=web_dev_agent,
    # asynchronous=True,
    input_data={"example_files_dir": example_web_files},
    asynchronous=False,
    tools=[DirectoryReadTool(), FileReadTool()]  # Add the necessary tools
)

In [111]:
# class ImageAnalysisResults(BaseModel):  # Create a new model to wrap the list
#     results: List[ImageAnalysisResult]



# critic_read_examples_task = read_examples_task
# critic_read_examples_task.agent = critic_full_agent   #need this here
#solves the following error:
# ValidationError: 1 validation error for Crew
#   Sequential process error: Agent is missing in the task with the following description: Read all the example files from the directory '/content/example-markdown'.
#                     Store the content of these files in your memory for later use. [type=missing_agent_in_task, input_value={'agents': [Agent(role=Ar...e': True, 'cache': True}, input_type=dict]

# critic_read_examples_task = read_examples_task.copy_with(agent=critic_full_agent)

write_image_web_content = Task(
    description=f"""You are provided with a list of objects.
                   Each object contains the following information about an image:
                   - filename
                   - description
                   - assessment
                   - quality_score
                   - date_taken
                   - title (currently None)
                   - summary (currently None)

                   Read the content of this list and store it in your memory.
                   For each image object, generate:
                   1. A compelling title based on the description and assessment.
                   2. A compelling summary incorporating key features and artistic qualities.

                   You make sure the title and summary you write are in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   Store the generated title and summary within the image object,
                   by updating the 'title' and 'summary' attributes.

                   You do not need to open any files. The list of objects is provided as input data here:
                   '{image_objects}'""",
    expected_output="""List of ImageAnalysisResult objects read, titles and summaries generated, and stored within the objects.""",
    agent=critic_full_agent,
    asynchronous=False,
    input_data={
          "image_objects": image_objects  # Pass the list of ImageAnalysisResult objects
    },
    context=[read_examples_task],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)

review_image_web_content = Task(
    description=f"""You perform a review of the each image title and image summary.
                    You make sure the title and summary are written in a similar style,
                   tone, voice, and length as in the example markdown files you have read.

                   The summary is describing the image only, with no reference to the artist himself,
                   as I am the artist and you are making sure the summary is well written on my behalf.
                   The summary should be written to be engaging, without seeming like it was written
                   by generative AI, i.e., use adjectives but do so without being excessive.

                   Make viewers and readers feel engaged and curious about the image, without tooting
                   my own horn too much. E.g., I would never describe my own composition as perfect,
                   I would be more subtle and descriptibe. This needs to be tasteful.

                   You make any changes you deem necessary to improve the text.""",
    expected_output="""List of ImageAnalysisResult objects read, titles and summaries generated, and stored within the objects.""",
    agent=critic_full_agent,
    asynchronous=False,
    # input_data={
    #       "image_objects": image_objects  # Pass the list of ImageAnalysisResult objects
    # },
    context=[write_image_web_content],
    # output_pydantic=ImageAnalysisResult #implement later, similar errors https://github.com/crewAIInc/crewAI/discussions/1436
)


In [112]:

process_all_subfolders_task = Task(
    description=f"""Navigate to the directory '{selected_dir}'.
                   For each subfolder in this directory:
                       1. Get the name of the current subfolder.
                       2. Find the corresponding image object in the '{image_objects}' list
                          by matching the subfolder name with the object's 'filename' attribute (without extension).
                       3. Open the file named 'index.md' in the subfolder. If it doesn't exist, create it.
                       4. Based on the example web markdown files you read earlier and the image object's data,
                          populate the 'index.md' file using the following attributes from the '{image_objects}':
                           - Title: Use the 'title' attribute from the image object.
                           - Date: Use the 'date_taken' attribute from the image object.
                           - Image Summary: Use the 'summary' attribute from the image object.
                           - Image: Ensure the image path in the markdown file is correct (e.g., '![](featured.jpg)').
                       5. Repeat steps 1-4 for all subfolders in '{selected_dir}'.
                   Use DirectoryReadTool to list files, FileReadTool to read files, and FileWriterTool to update/create the 'index.md' files.
                   It is important that you do not rewrite the image summary or the image title. You should use the
                   summary and the title you received from the Art Critic Agent""",
    expected_output=f"""'index.md' files updated/created in all subfolders of '{selected_dir}' with content from image objects.
                    Characters aligned with UTF-8 standards and best practices.
                    the file must include the attributes:
                    "title: "
                    "authors:
                      -admin"
                    "tags:
                      - Photography"
                    "image:
                      placement: 2
                      preview_only: false"
                    """,
    agent=web_dev_agent,
    context=[read_examples_task, review_image_web_content],
    tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],  # Add the necessary tools
    asynchronous=False,
    input_data={
        "selected_dir": selected_dir,
        # "image_objects": image_objects,
        "example_files_dir": example_web_files
    }
)

# review_index_structure = Task(
#     description=f"""You open the folder '{selected_dir}'.
#                     In this folder there are multiple sub folders, each named after an image.
#                     You iterate through each sub folder. You read the index.md files in each subfolder
#                     and compare the file to the example markdown files you previously read
#                     in the task 'read_examples_task'.

#                     You verify that the technical structure of the index.md files is correct,
#                     and the technical structure is identical to the example index markdown files.

#                     This includes verifying the correct variables, date structure, and similar.
#                     You DO NOT change any of the content itself, which is the title and the image description.
#                     """,
#     expected_output=f"""'index.md' files updated in all subfolders of '{selected_dir}' with content from image objects.
#                     The title and the image description/summary will NOT be changed by you.
#                     The file must include the attributes:
#                     "title: "
#                     "authors:
#                       -admin"
#                     "tags:
#                       - Photography"
#                     "image:
#                       placement: 2
#                       preview_only: false"
#                     """,
#     agent=web_dev_agent,
#     asynchronous=False,
#     context=[read_examples_task, process_all_subfolders_task],
#     tools=[DirectoryReadTool(), FileReadTool(), FileWriterTool()],
#     input_data={
#         "selected_dir": selected_dir
#     }
# )

In [113]:
full_crew = Crew(
    agents=[critic_full_agent,
            web_dev_agent
            ],
    tasks=[
           read_examples_task,
           write_image_web_content,
           review_image_web_content,
           process_all_subfolders_task,
          #  review_index_structure
           ],
    process=Process.sequential,
    verbose=True,
    # memory=True,
    cache = True,
)

results = full_crew.kickoff()
print(results)

2024-12-31 16:14:23,804 - 47584 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Web Developer
## Task: Read all the example files from the directory 'example-files/example-markdown/'.
                    Store the content of these files in your memory for later use.


# Agent: Web Developer
## Using tool: List files in directory
## Tool Input: 
"{\"directory\": \"example-files/example-markdown/\"}"
## Tool Output: 
File paths: 
-example-files/example-markdown/index1.md
- example-files/example-markdown/index2.md
- example-files/example-markdown/index3.md


# Agent: Web Developer
## Using tool: Read a file's content
## Tool Input: 
"{\"file_path\": \"example-files/example-markdown/index1.md\"}"
## Tool Output: 
---
title: 'Roadrunner in Big Bend National Park'
authors:
- admin
tags:
- Photography
date: "2023-04-06T00:00:00Z"
featured: false
draft: false

image:
  placement: 2
  focal_point: ""
  preview_only: false
---

A Greater Roadrunner stands alert on a rocky outcrop in Big Bend National Park, Texas. The rugged mountains provide a stunning backdrop to 

In [114]:
import chardet

def ensure_utf8_encoding(selected_dir):
    """
    Recursively processes all Markdown files in the selected directory and
    its subdirectories, ensuring they are encoded in UTF-8.
    
    Args:
        selected_dir (str): Path to the folder containing Markdown files.
    """
    for root, dirs, files in os.walk(selected_dir):
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                # Read the file and detect its current encoding
                with open(file_path, 'rb') as f:
                    raw_data = f.read()
                    detected = chardet.detect(raw_data)
                    current_encoding = detected['encoding']
                
                # If not UTF-8, convert and overwrite
                if current_encoding.lower() != 'utf-8':
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(raw_data.decode(current_encoding))
                    print(f"Converted {file_path} to UTF-8.")
                else:
                    print(f"{file_path} is already UTF-8.")

# Example usage
ensure_utf8_encoding(selected_dir)

Converted example-files/selected-pics/P9021062\index.md to UTF-8.
Converted example-files/selected-pics/P9021081\index.md to UTF-8.


In [115]:
# Now you can access the attributes of the objects:
# for obj in image_objects:
#     print(f"Filename: {obj.filename}")
#     print(f"Description: {obj.description}")
#     print(f"Assessment: {obj.assessment}")
#     print(f"Quality Score: {obj.quality_score}")
#     print(f"Date Taken: {obj.date_taken}")
#     print(obj.title)  # Access the generated title
#     print(obj.summary) # Access the generated summary
#     print("-" * 20)